In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import pandas as pd

# Ruta al archivo JSON
json_file_path = "/content/drive/MyDrive/datasets_originales/EXIST 2024 Tweets Dataset/training/EXIST2024_training.json"

# Crear una tabla para almacenar los datos
columns = ["Annotator", "Gender", "Age", "Ethnicity", "Study Level", "Country", "Confidence"]
annotator_table = pd.DataFrame(columns=columns)

# Leer el dataset desde el archivo JSON
with open(json_file_path, 'r', encoding='utf-8') as file:
    dataset = json.load(file)

# Iterar sobre el dataset
for tweet_id, tweet_data in dataset.items():
    annotators = tweet_data["annotators"]
    for annotator in annotators:
        # Verificar si el annotator ya está en la tabla
        if annotator not in annotator_table["Annotator"].values:
            # Obtener los datos del annotator
            gender = tweet_data["gender_annotators"][annotators.index(annotator)]
            age = tweet_data["age_annotators"][annotators.index(annotator)]
            ethnicity = tweet_data["ethnicities_annotators"][annotators.index(annotator)]

            # Verificar si el nivel de estudio está en el diccionario
            study_level = tweet_data["study_levels_annotators"][annotators.index(annotator)]
            study_level_dict = {
                "Less than high school diploma": 1,
                "High school degree or equivalent": 2,
                "Bachelor’s degree": 3,
                "Master’s degree": 4,
                "Doctorate": 5
            }
            # study_level_score = study_level_dict.get(study_level, 0)  # Si no está en el diccionario, asignar 0

            country = tweet_data["countries_annotators"][annotators.index(annotator)]

            # Calcular la puntuación de confianza
            age_score = {"18-22": 1, "23-45": 2, "46+": 3}.get(age, 0)  # Si no está en el diccionario, asignar 0
            gender_score = {"F": 1, "M": 0.5}.get(gender, 0)  # Si no está en el diccionario, asignar 0

            study_level_score = study_level_dict.get(study_level, 0)  # Si no está en el diccionario, asignar 0

            confidence = age_score * 0.3 + gender_score * 0.2 + study_level_score * 0.5


            # Agregar el annotator a la tabla
            annotator_table = annotator_table.append({
                "Annotator": annotator,
                "Gender": gender,
                "Age": age,
                "Ethnicity": ethnicity,
                "Study Level": study_level,
                "Country": country,
                "Confidence": confidence
            }, ignore_index=True)

# Mostrar las primeras filas de la tabla resultante
print(annotator_table.head())


<ipython-input-4-ecba6a6ac123>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  annotator_table = annotator_table.append({
<ipython-input-4-ecba6a6ac123>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  annotator_table = annotator_table.append({
<ipython-input-4-ecba6a6ac123>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  annotator_table = annotator_table.append({
<ipython-input-4-ecba6a6ac123>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  annotator_table = annotator_table.append({
<ipython-input-4-ecba6a6ac123>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

     Annotator Gender    Age           Ethnicity  \
0  Annotator_1      F  18-22  White or Caucasian   
1  Annotator_2      F  23-45   Hispano or Latino   
2  Annotator_3      F    46+  White or Caucasian   
3  Annotator_4      M    46+  White or Caucasian   
4  Annotator_5      M  23-45  White or Caucasian   

                        Study Level        Country  Confidence  
0                 Bachelor’s degree          Italy         2.0  
1                 Bachelor’s degree         Mexico         2.3  
2  High school degree or equivalent  United States         2.1  
3                   Master’s degree          Spain         3.0  
4                   Master’s degree          Spain         2.7  


In [ ]:
print(annotator_table)

         Annotator Gender    Age           Ethnicity  \
0      Annotator_1      F  18-22  White or Caucasian   
1      Annotator_2      F  23-45   Hispano or Latino   
2      Annotator_3      F    46+  White or Caucasian   
3      Annotator_4      M    46+  White or Caucasian   
4      Annotator_5      M  23-45  White or Caucasian   
..             ...    ...    ...                 ...   
720  Annotator_721      F  23-45  White or Caucasian   
721  Annotator_722      M  18-22   Hispano or Latino   
722  Annotator_723      M  23-45  White or Caucasian   
723  Annotator_724      M    46+  White or Caucasian   
724  Annotator_725      F    46+  White or Caucasian   

                          Study Level         Country  Confidence  
0                   Bachelor’s degree           Italy         2.0  
1                   Bachelor’s degree          Mexico         2.3  
2    High school degree or equivalent   United States         2.1  
3                     Master’s degree           Spain  

In [ ]:
import json
import pandas as pd
from collections import Counter

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Ruta al archivo JSON
json_file_path = "/content/drive/MyDrive/datasets_originales/EXIST 2024 Tweets Dataset/dev/EXIST2024_dev.json"

# Leer el dataset desde el archivo JSON
with open(json_file_path, 'r', encoding='utf-8') as file:
    dataset = json.load(file)

# Crear una tabla para almacenar los datos finales
columns_final = ["id_EXIST", "lang", "tweet", "label_task2", "split"]
final_table = pd.DataFrame(columns=columns_final)

# Crear una tabla para almacenar información sobre los empates
columns_empates = ["id_EXIST", "posibles_soluciones", "solucion_elegida"]
empates_table = pd.DataFrame(columns=columns_empates)

# Contador para el número de empates
empates_count = 0

# Iterar sobre el dataset
for tweet_id, tweet_data in dataset.items():
    annotators = tweet_data["annotators"]
    label_task2_votes = Counter(tweet_data["labels_task2"])

    # Manejar el empate y seleccionar la votación con mayor "Confidence" en caso de empate
    max_label_task2 = max(label_task2_votes, key=lambda k: (label_task2_votes[k], sum(annotator_table[annotator_table["Annotator"].isin(annotators)]["Confidence"])))

    # Convertir "-" a "NO"
    max_label_task2 = "NO" if max_label_task2 == "-" else max_label_task2

    # Si max_label_task2 es "UNKNOWN", omitir este documento
    if max_label_task2 == "UNKNOWN":
        continue

    # Obtener otros campos
    id_exist = tweet_data["id_EXIST"]
    lang = tweet_data["lang"]
    tweet = tweet_data["tweet"]
    split = tweet_data["split"]

    # Agregar la entrada a la tabla final
    final_table = final_table.append({
        "id_EXIST": id_exist,
        "lang": lang,
        "tweet": tweet,
        "label_task2": max_label_task2,
        "split": split
    }, ignore_index=True)

    # Verificar si hay empate
    if list(label_task2_votes.values()).count(label_task2_votes[max_label_task2]) > 1:
        empates_count += 1
        posibles_soluciones = [label for label, count in label_task2_votes.items() if count == label_task2_votes[max_label_task2]]
        solucion_elegida = max(posibles_soluciones, key=lambda label: sum(annotator_table[annotator_table["Annotator"].isin(annotators) & (final_table["label_task2"] == label)]["Confidence"]))

        # Agregar información sobre el empate a la tabla de empates
        empates_table = empates_table.append({
            "id_EXIST": id_exist,
            "posibles_soluciones": posibles_soluciones,
            "solucion_elegida": solucion_elegida
        }, ignore_index=True)

# Mostrar las primeras filas de la tabla final
print(final_table.head())

# Imprimir el número de empates
print(f"Número de empates: {empates_count}")

# Mostrar la tabla de empates
print(empates_table.head())

<ipython-input-8-eb8585609122>:60: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  solucion_elegida = max(posibles_soluciones, key=lambda label: sum(annotator_table[annotator_table["Annotator"].isin(annotators) & (final_table["label_task2"] == label)]["Confidence"]))
<ipython-input-8-eb8585609122>:60: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  solucion_elegida = max(posibles_soluciones, key=lambda label: sum(annotator_table[annotator_table["Annotator"].isin(annotators) & (final_table["label_task2"] == label)]["Confidence"]))
<ipython-input-8-eb8585609122>:60: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  solucion_elegida = max(posibles_soluciones, key=lambda label: sum(annotator_table[annotator_table["Annotator"].isin(annotators) & (final_table["label_task2"] == label)]["Confidence"]))
<ipython-input-8-eb8585609122>:60: UserWarning: Boolean Series key will be reindexed to match DataFrame in

  id_EXIST lang                                              tweet  \
0   300001   es  @Fichinescu La comunidad gamer es un antro de ...   
1   300002   es  @anacaotica88 @MordorLivin No me acuerdo de lo...   
2   300003   es  @cosmicJunkBot lo digo cada pocos dias y lo re...   
3   300004   es  Also mientras les decia eso la señalaba y deci...   
4   300005   es  And all people killed,  attacked, harassed by ...   

   label_task2   split  
0           NO  DEV_ES  
1  JUDGEMENTAL  DEV_ES  
2           NO  DEV_ES  
3     REPORTED  DEV_ES  
4           NO  DEV_ES  
Número de empates: 86
  id_EXIST         posibles_soluciones solucion_elegida
0   300020  [JUDGEMENTAL, REPORTED, -]      JUDGEMENTAL
1   300025     [REPORTED, JUDGEMENTAL]         REPORTED
2   300034               [REPORTED, -]         REPORTED
3   300036       [REPORTED, -, DIRECT]         REPORTED
4   300038       [JUDGEMENTAL, DIRECT]      JUDGEMENTAL


In [ ]:
# Guardar la tabla final en CSV
final_table.to_csv('/content/drive/MyDrive/DATASETS_BASELINE/test_data.csv', index=False, encoding='utf-8')

# Guardar la tabla de empates en CSV
empates_table.to_csv('/content/drive/MyDrive/DATASETS_BASELINE/test_empates_table.csv', index=False, encoding='utf-8')